# 02 - Feature Engineering for Impervious Surfaces in Göteborg

The purpose of this notebook is to compute engineered features for each impervious surface polygon created in **Notebook 01**. These features will later support:

- Suitability scoring (environmental, social, spatial indicators)  
- Clustering and typology development for sealed surfaces  
- A lightweight ML classifier (Notebook 04) to predict “green → artificial” transitions  

We prioritise features that are:

- Meaningful for the **Grey to Green** initiative  
- Computationally efficient  

---

## Core Feature Groups

### **1. Geometry-based features**

- Area (m², ha)  
- Perimeter  
- Shape compactness index  
- Elongation / aspect ratio  

These help distinguish small parking lots from industrial slabs, residential courtyards, and major arterials.

---

### **2. Proximity features**

Distances computed from each impervious polygon to:

- Nearest existing **green space** (UA2018)  
- Nearest **water body** (UA2018)  
- Nearest **major road** (OSM)  
- **City centre** (Gustav Adolfs Torg, reference point)  

These indicators support suitability and prioritisation scoring.

---

### **3. Context features**

Local environmental and urban context within a fixed buffer (e.g., 100 m):

- Land-cover mix within the buffer  
- Nearest neighbourhood ID (stadsområden, mellanområden)  
- Population density or estimates (UA2018 Pop2018, or population grid joins)  

These variables help capture how each surface sits within its surrounding urban fabric.

---

## Output

All engineered features will be stored in:

- *data/processed/impervious_features.gpkg*

This processed GeoDataFrame serves as the input for:

- **Notebook 03:** Suitability Scoring  
- **Notebook 04:** ML classifier predicting future sealing patterns  

By enriching each impervious polygon with spatial, geometric, and contextual indicators, this notebook establishes the feature foundation for the next stages of the Grey to Green analysis.




In [4]:
import pathlib

import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Point
from shapely.ops import nearest_points
import matplotlib.pyplot as plt

# Base directory
PROJECT_DIR = pathlib.Path("..").resolve()

DATA_RAW = PROJECT_DIR / "data" / "raw"
DATA_PROCESSED = PROJECT_DIR / "data" / "processed"
OUTPUTS = PROJECT_DIR / "outputs"
MAPS_DIR = OUTPUTS / "maps"

PROJECT_DIR, DATA_RAW, DATA_PROCESSED, MAPS_DIR

# Main processed impervious dataset from Notebook 01
impervious_path = DATA_PROCESSED / "impervious_current.gpkg"

impervious = gpd.read_file(impervious_path)

# Boundary 
stads_path = DATA_RAW / "Stadsområde_shp" / "Jur_stadsomraden_xu_region.shp" 
stadsomraden = gpd.read_file(stads_path)
gothenburg_boundary = stadsomraden.dissolve()

impervious.head()


,source,area_m2,area_ha,geometry
0,ua2018,3732.744159,0.373274,"MULTIPOLYGON Z (((146950.814 6382814.598 0, 14..."
1,ua2018,5257.072176,0.525707,"MULTIPOLYGON Z (((147008.797 6382955.981 0, 14..."
2,ua2018,3540.900042,0.354090,"MULTIPOLYGON Z (((146878.882 6383060.99 0, 146..."
3,ua2018,23052.689464,2.305269,"MULTIPOLYGON Z (((146960.807 6383417.447 0, 14..."
4,ua2018,107752.964736,10.775296,"MULTIPOLYGON Z (((147220.719 6383259.446 0, 14..."


## 1. Geometry Features

In [5]:
# Load processed layers for features

# UA2018 clipped land cover 
ua2018_clipped_path = DATA_PROCESSED / "ua2018_clipped.gpkg"
ua2018_clipped = gpd.read_file(ua2018_clipped_path)

# UA2018 impervious
ua2018_artificial_path = DATA_PROCESSED / "ua2018_artificial.gpkg"
ua2018_artificial = gpd.read_file(ua2018_artificial_path)

# OSM-derived road and parking polygons
osm_road_path = DATA_PROCESSED / "osm_road_polygons.gpkg"
osm_road_polygons = gpd.read_file(osm_road_path)

osm_parking_path = DATA_PROCESSED / "osm_parking.gpkg"
osm_parking = gpd.read_file(osm_parking_path)

# Make sure everything is in the same CRS as impervious (should be EPSG:3007)
print("impervious CRS:", impervious.crs)
print("ua2018_clipped CRS:", ua2018_clipped.crs)
print("ua2018_artificial CRS:", ua2018_artificial.crs)
print("osm_road_polygons CRS:", osm_road_polygons.crs)
print("osm_parking CRS:", osm_parking.crs)

# Reproject if needed (defensive, in case any layer was changed)
target_crs = impervious.crs

ua2018_clipped = ua2018_clipped.to_crs(target_crs)
ua2018_artificial = ua2018_artificial.to_crs(target_crs)
osm_road_polygons = osm_road_polygons.to_crs(target_crs)
osm_parking = osm_parking.to_crs(target_crs)

ua2018_clipped.head()


impervious CRS: EPSG:3007
ua2018_clipped CRS: EPSG:3007
ua2018_artificial CRS: EPSG:3007
osm_road_polygons CRS: EPSG:3007
osm_parking CRS: EPSG:3007


,country,fua_name,fua_code,code_2018,class_2018,prod_date,identifier,perimeter,area,comment,Pop2018,code_2018_str,geometry
0,SE,Göteborg,SE002L1,11220,Discontinuous medium density urban fabric (S.L...,2020-07,1348-SE002L1,277.683299,3732.743332,None,22,11220,"MULTIPOLYGON Z (((146950.814 6382814.598 0, 14..."
1,SE,Göteborg,SE002L1,32000,Herbaceous vegetation associations (natural gr...,2020-07,54211-SE002L1,844.144050,16673.972611,None,0,32000,"MULTIPOLYGON Z (((147173.984 6382938.908 0, 14..."
2,SE,Göteborg,SE002L1,11220,Discontinuous medium density urban fabric (S.L...,2020-07,1347-SE002L1,514.391492,5257.070782,None,38,11220,"MULTIPOLYGON Z (((147008.797 6382955.981 0, 14..."
3,SE,Göteborg,SE002L1,14200,Sports and leisure facilities,2020-07,35342-SE002L1,238.742986,2716.533347,None,0,14200,"MULTIPOLYGON Z (((147103.796 6382989.655 0, 14..."
4,SE,Göteborg,SE002L1,11230,Discontinuous low density urban fabric (S.L. :...,2020-07,3683-SE002L1,308.005411,3540.899052,None,21,11230,"MULTIPOLYGON Z (((146878.882 6383060.99 0, 146..."


In [6]:
# Basic geometry-based features for impervious polygons

# Area and perimeter
impervious["area_m2"] = impervious.geometry.area
impervious["area_ha"] = impervious["area_m2"] / 10_000
impervious["perimeter_m"] = impervious.geometry.length

# Shape compactness: perimeter² / (4π area) – higher = less compact
impervious["compactness"] = (
    impervious["perimeter_m"] ** 2
) / (4 * np.pi * impervious["area_m2"].replace(0, np.nan))

# Bounding box aspect ratio: width / height
def bbox_ratio(geom):
    minx, miny, maxx, maxy = geom.bounds
    width = maxx - minx
    height = maxy - miny
    return width / (height + 1e-9)

impervious["bbox_ratio"] = impervious.geometry.apply(bbox_ratio)

impervious[["source", "area_ha", "perimeter_m", "compactness", "bbox_ratio"]].head()


,source,area_ha,perimeter_m,compactness,bbox_ratio
0,ua2018,0.373274,277.693739,1.643971,0.836845
1,ua2018,0.525707,514.365502,4.004883,0.976495
2,ua2018,0.354090,308.162464,2.134204,0.415632
3,ua2018,2.305269,1128.530042,4.396376,0.202775
4,ua2018,10.775296,4361.349415,14.047617,0.846141


## 2. Proximity Features

In [10]:
from shapely.ops import nearest_points

def compute_nearest_distance(source_gdf, target_gdf, source_label):
    """
    Efficient nearest-neighbour distance computation using spatial index.
    Computes distance from each geometry in source_gdf to nearest geometry in target_gdf.
    Returns a NumPy array of distances in meters.
    """
    # Build spatial index for targets
    target_sindex = target_gdf.sindex

    distances = []

    for geom in source_gdf.geometry:
        if geom is None or geom.is_empty:
            distances.append(np.nan)
            continue

        # Query nearest target using geometry (not bounds)
        possible_matches_index = list(target_sindex.nearest(geom, return_all=False))
        nearest_idx = possible_matches_index[0]

        nearest_geom = target_gdf.iloc[nearest_idx].geometry

        # Compute precise distance
        dist = geom.distance(nearest_geom)
        distances.append(dist)

    print(f"Computed distances: {source_label}")
    return np.array(distances)


In [11]:
# Ensure supporting layers exist and are projected correctly
ua2018_clipped["code_2018_str"] = ua2018_clipped["code_2018"].astype(str)

# Green classes
ua_green_codes = ["14100", "14200", "21000", "23000", "31000", "32000"]
ua_green = ua2018_clipped[ua2018_clipped["code_2018_str"].isin(ua_green_codes)].copy()

# Water classes
ua_water_codes = ["50000"]
ua_water = ua2018_clipped[ua2018_clipped["code_2018_str"].isin(ua_water_codes)].copy()

# Use centroids for distance
ua_green_centroids = ua_green.copy()
ua_green_centroids["geometry"] = ua_green_centroids.geometry.centroid

ua_water_centroids = ua_water.copy()
ua_water_centroids["geometry"] = ua_water_centroids.geometry.centroid

print(len(ua_green_centroids), "green centroids")
print(len(ua_water_centroids), "water centroids")

3302 green centroids
133 water centroids


In [12]:
# Distance to nearest green
impervious["dist_to_green_m"] = compute_nearest_distance(
    impervious, ua_green_centroids, "dist_to_green"
)

# Distance to nearest water
impervious["dist_to_water_m"] = compute_nearest_distance(
    impervious, ua_water_centroids, "dist_to_water"
)

Computed distances: dist_to_green
Computed distances: dist_to_water


In [13]:
# Define city centre in WGS84 → transform to CRS 3007
city_center_lonlat = gpd.GeoDataFrame(
    geometry=[Point(11.967, 57.707)], crs="EPSG:4326"
).to_crs(impervious.crs)

city_center_point = city_center_lonlat.geometry.iloc[0]

impervious["dist_to_city_center_m"] = impervious.geometry.apply(
    lambda g: g.distance(city_center_point)
)


In [14]:
# Use centroids for road polygons 
osm_road_centroids = osm_road_polygons.copy()
osm_road_centroids["geometry"] = osm_road_centroids.geometry.centroid

impervious["dist_to_major_road_m"] = compute_nearest_distance(
    impervious, osm_road_centroids, "dist_to_major_road"
)


Computed distances: dist_to_major_road


In [15]:
impervious[[
    "dist_to_green_m",
    "dist_to_water_m",
    "dist_to_city_center_m",
    "dist_to_major_road_m"
]].describe()


,dist_to_green_m,dist_to_water_m,dist_to_city_center_m,dist_to_major_road_m
count,112903.000000,112903.000000,112903.000000,112903.000000
mean,16820.646732,15396.967441,6425.484327,19194.405141
std,5820.005749,5374.953303,4043.629432,5975.950984
min,55.715476,76.909641,0.000000,128.625440
25%,13221.075449,11898.091979,3178.807731,15332.947912
50%,16609.999212,14841.816024,5436.586957,18634.341684
75%,20646.504086,19137.968321,9166.926718,22641.540940
max,34968.798182,32253.279958,26134.655916,39569.869485


## 3. Context Feature

In [16]:
# Check what columns exist in stadsomraden
stadsomraden.columns


Index(['SW_MEMBER', 'NAMN', 'AREA_KM2', 'STADSOMRAD', 'REG_DATUM',
       'AJOUR_DATU', 'OPERATÖR', 'geometry'],
      dtype='object')

In [17]:
stads_cols_to_keep = ["NAMN", "STADSOMRAD", "geometry"]

stads_for_join = stadsomraden[stads_cols_to_keep].copy()

# Spatial join: assign each impervious polygon to a stadsområde
impervious_with_stads = gpd.sjoin(
    impervious,
    stads_for_join,
    how="left",
    predicate="intersects"
)

impervious_with_stads.head()


,source,area_m2,area_ha,geometry,perimeter_m,compactness,bbox_ratio,dist_to_green_m,dist_to_water_m,dist_to_city_center_m,dist_to_major_road_m,index_right,NAMN,STADSOMRAD
0,ua2018,3732.744159,0.373274,"MULTIPOLYGON Z (((146950.814 6382814.598 0, 14...",277.693739,1.643971,0.836845,153.472944,3510.938743,15786.136616,9836.450600,1,Sydväst,3.0
1,ua2018,5257.072176,0.525707,"MULTIPOLYGON Z (((147008.797 6382955.981 0, 14...",514.365502,4.004883,0.976495,168.673464,3443.009132,15695.898183,9822.506185,1,Sydväst,3.0
2,ua2018,3540.900042,0.354090,"MULTIPOLYGON Z (((146878.882 6383060.99 0, 146...",308.162464,2.134204,0.415632,326.246045,3438.818296,15510.553562,9746.034351,1,Sydväst,3.0
3,ua2018,23052.689464,2.305269,"MULTIPOLYGON Z (((146960.807 6383417.447 0, 14...",1128.530042,4.396376,0.202775,217.247102,3226.591225,15216.656950,9783.604491,1,Sydväst,3.0
4,ua2018,107752.964736,10.775296,"MULTIPOLYGON Z (((147220.719 6383259.446 0, 14...",4361.349415,14.047617,0.846141,216.436698,2942.794147,15172.776695,9889.374023,1,Sydväst,3.0


In [18]:
# Final feature dataset for modelling
features_path = DATA_PROCESSED / "impervious_features.gpkg"

impervious_with_stads.to_file(features_path, driver="GPKG")
print("Saved feature layer to:", features_path)


Saved feature layer to: D:\Programming\geo-projects\impervious-to-green-gbg\data\processed\impervious_features.gpkg


In [19]:
# Quick overview of the engineered features

impervious_with_stads[[
    "source",
    "area_ha",
    "perimeter_m",
    "compactness",
    "bbox_ratio",
    "dist_to_green_m",
    "dist_to_water_m",
    "dist_to_city_center_m",
    "dist_to_major_road_m",
    "NAMN",
    "STADSOMRAD",
]].describe(include="all")


,source,area_ha,perimeter_m,compactness,bbox_ratio,dist_to_green_m,dist_to_water_m,dist_to_city_center_m,dist_to_major_road_m,NAMN,STADSOMRAD
count,113232,113232.000000,1.132320e+05,113232.000000,113232.000000,113232.000000,113232.000000,113232.000000,113232.000000,113232,113232.000000
unique,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN
top,osm_road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hisingen,NaN
freq,101083,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40439,NaN
mean,NaN,0.265619,3.620213e+02,5.692008,1.318359,16815.571046,15390.903683,6417.113261,19191.981637,NaN,2.805885
std,NaN,11.605530,2.437051e+04,410.166002,1.333464,5814.667293,5370.093917,4041.366150,5970.777973,NaN,1.067683
min,NaN,0.000057,4.132430e+00,1.000315,0.025224,55.715476,76.909641,0.000000,128.625440,NaN,1.000000
25%,NaN,0.020547,5.756498e+01,1.274448,0.619959,13218.159647,11896.990160,3175.434517,15317.987606,NaN,2.000000
50%,NaN,0.040903,9.790025e+01,1.789586,0.994863,16610.308513,14835.172213,5420.852765,18635.050490,NaN,3.000000
75%,NaN,0.103939,2.166299e+02,3.210306,1.570436,20636.194917,19126.121197,9153.310257,22633.661734,NaN,4.000000


# Notebook 02 — Feature Engineering for Impervious Surfaces

In this notebook we transformed the unified impervious baseline from Notebook 01 into a
feature-rich dataset suitable for spatial analysis, suitability scoring, and machine learning.

## 1. Inputs

We loaded the following processed layers from `data/processed/`:

- `impervious_current.gpkg` — unified impervious polygons (UA2018 + OSM roads + OSM parking)
- `ua2018_clipped.gpkg` — full Urban Atlas 2018 land cover within Göteborg
- `ua2018_artificial.gpkg` — Urban Atlas impervious subset
- `osm_road_polygons.gpkg` — buffered road polygons derived from OSM
- `osm_parking.gpkg` — parking polygons from OSM
- `Jur_stadsomraden_xu_region.shp` — Göteborg’s official *stadsområden* (city districts)

All layers were reprojected to a common projected CRS (SWEREF99 12 00, EPSG:3007), ensuring that
distances and areas are expressed in metres and square metres.

## 2. Geometry-Based Features

For each impervious polygon we computed basic shape descriptors:

- **Area** (`area_m2`, `area_ha`)
- **Perimeter** (`perimeter_m`)
- **Compactness**: \\( \text{perimeter}^2 / (4 \pi \cdot \text{area}) \\),
  where higher values indicate more elongated or irregular shapes.
- **Bounding-box aspect ratio** (`bbox_ratio`), approximating elongation.

These features already distinguish between different types of sealed surfaces
(e.g. narrow roads vs. compact parking lots vs. large industrial blocks).

## 3. Distance-Based Features

Using spatial indexes and centroids, we computed distances from each impervious polygon to:

- **Nearest green area** (`dist_to_green_m`), using Urban Atlas classes:
  urban green areas, sports/leisure, forests, pastures, arable land, and herbaceous vegetation.
- **Nearest water body** (`dist_to_water_m`), using the Urban Atlas water class.
- **City centre** (`dist_to_city_center_m`), based on a reference point near Gustav Adolfs Torg.
- **Nearest major road** (`dist_to_major_road_m`), using centroids of buffered OSM road polygons.

All distances are stored in metres. These features capture how “embedded” each impervious surface is
in the urban structure and its proximity to existing green/blue infrastructure and mobility corridors.

## 4. Context Features: Stadsområden

We performed a spatial join between the impervious polygons and Göteborg’s official
*stadsområden* layer, attaching:

- **`NAMN`** — name of the stadsområde
- **`STADSOMRAD`** — stadsområde code

This provides a direct link between each impervious surface and its administrative context,
enabling aggregations and comparisons between different parts of the city.

## 5. Output

The final feature dataset was saved as:

- `data/processed/impervious_features.gpkg`

This GeoPackage contains, for each impervious polygon:

- Source information (`ua2018`, `osm_road`, `osm_parking`)
- Geometry-based features (area, perimeter, compactness, bbox_ratio)
- Distance features (to green, water, city centre, major roads)
- Stadsområde attributes (`NAMN`, `STADSOMRAD`)

## 6. Role in the Overall Project

This engineered feature table is the core input for the next steps:

- **Notebook 03:** Suitability scoring and basic typologies of impervious surfaces.
- **Notebook 04:** Small ML experiment/classifier to understand which surfaces are more likely
  to be associated with “green → artificial” transitions or high greening potential.
- **Notebook 05:** Prioritisation maps and summary statistics for communicating results.

Together with the baseline maps from Notebook 01, this notebook turns raw geospatial data
into structured, model-ready information that can support the *Grey to Green* agenda:
identifying where Göteborg’s grey infrastructure could most effectively be transformed into
new green and blue spaces.
